In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
from PIL import Image, ImageTk
import fitz  # PyMuPDF
import re
import requests
from transformers import pipeline
from keybert import KeyBERT
from datetime import datetime, timedelta
from joblib import dump, load
import os
import torch
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns
import time
import random
import threading
from typing import Dict, List

# ========== Modern Color Scheme ==========
DARK_BLUE = "#1a237e"
LIGHT_BLUE = "#3949ab"
ACCENT_BLUE = "#00b0ff"
WHITE = "#ffffff"
LIGHT_GRAY = "#f5f5f5"
DARK_GRAY = "#212121"
GREEN = "#4caf50"
ORANGE = "#ff9800"
RED = "#f44336"

# ========== Font Styles ==========
FONT_LARGE = ("Segoe UI", 16, "bold")
FONT_MEDIUM = ("Segoe UI", 12)
FONT_SMALL = ("Segoe UI", 10)
FONT_TITLE = ("Segoe UI", 20, "bold")
FONT_SUBTITLE = ("Segoe UI", 12, "italic")

# ========== Enhanced Skill Database ==========
SKILL_DATABASE = {
    'Python': {'category': 'Programming', 'demand': 'Very High', 'weight': 1.2},
    'Java': {'category': 'Programming', 'demand': 'High', 'weight': 1.1},
    'C++': {'category': 'Programming', 'demand': 'High', 'weight': 1.1},
    'C': {'category': 'Programming', 'demand': 'Medium', 'weight': 1.0},
    'JavaScript': {'category': 'Web Development', 'demand': 'Very High', 'weight': 1.2},
    'MATLAB': {'category': 'Scientific Computing', 'demand': 'Medium', 'weight': 0.9},
    'Machine Learning': {'category': 'AI/Data Science', 'demand': 'Very High', 'weight': 1.3},
    'Data Science': {'category': 'AI/Data Science', 'demand': 'Very High', 'weight': 1.3},
    'Data Analysis': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.1},
    'Deep Learning': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.2},
    'Supervised Learning': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.1},
    'Unsupervised Learning': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.1},
    'Feature Engineering': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.1},
    'Object-Oriented Programming': {'category': 'Software Development', 'demand': 'High', 'weight': 1.1},
    'Data Structures': {'category': 'Software Development', 'demand': 'High', 'weight': 1.1},
    'Algorithms': {'category': 'Software Development', 'demand': 'High', 'weight': 1.1},
    'Embedded Systems': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'IoT': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'Arduino': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'ESP32': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'Adafruit IO': {'category': 'Embedded/IoT', 'demand': 'Low', 'weight': 0.8},
    'Sensors': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'Actuators': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'Circuit Design': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'Microcontrollers': {'category': 'Embedded/IoT', 'demand': 'Medium', 'weight': 1.0},
    'MySQL': {'category': 'Databases', 'demand': 'High', 'weight': 1.1},
    'Oracle': {'category': 'Databases', 'demand': 'Medium', 'weight': 1.0},
    'SQL': {'category': 'Databases', 'demand': 'High', 'weight': 1.1},
    'TensorFlow': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.2},
    'PyTorch': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.2},
    'Pandas': {'category': 'Data Analysis', 'demand': 'High', 'weight': 1.1},
    'NumPy': {'category': 'Data Analysis', 'demand': 'High', 'weight': 1.1},
    'Scikit-learn': {'category': 'AI/Data Science', 'demand': 'High', 'weight': 1.1},
}

# ========== Load Models ==========
def load_models():
    """Load or initialize ML models with enhanced error handling"""
    os.makedirs('models', exist_ok=True)
    
    try:
        if os.path.exists('models/kw_model.joblib'):
            kw_model = load('models/kw_model.joblib')
        else:
            kw_model = KeyBERT(model="paraphrase-MiniLM-L3-v2")
            dump(kw_model, 'models/kw_model.joblib')
            
        if os.path.exists('models/summarizer.joblib'):
            summarizer = load('models/summarizer.joblib')
        else:
            summarizer = pipeline("summarization", 
                                model="sshleifer/distilbart-cnn-12-6", 
                                device=0 if torch.cuda.is_available() else -1)
            dump(summarizer, 'models/summarizer.joblib')
            
        return kw_model, summarizer
        
    except Exception as e:
        messagebox.showerror("Model Loading Error", f"Error loading models: {e}")
        # Fallback to simpler models if primary ones fail
        return KeyBERT(), pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

KW_MODEL, SUMMARIZER = load_models()

# ========== Core Analysis Functions ==========
@lru_cache(maxsize=5)
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extract text content from PDF resume with error handling"""
    try:
        with fitz.open(pdf_path) as doc:
            return "\n".join(page.get_text() for page in doc)
    except Exception as e:
        messagebox.showerror("PDF Extraction Error", f"Error extracting text from PDF: {e}")
        return ""

def extract_contact_info(text: str) -> Dict[str, str]:
    """Enhanced contact information extraction with more patterns"""
    email = re.search(r'\b[\w.-]+@[\w.-]+\.\w+\b', text)
    phone = re.search(r'\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b', text)
    linkedin = re.search(r'(?:https?://)?(?:www\.)?linkedin\.com/(?:in|pub|company)/[a-zA-Z0-9-]+', text)
    github = re.search(r'(?:https?://)?(?:www\.)?github\.com/[a-zA-Z0-9-]+', text)
    
    return {
        'email': email.group() if email else "Not found",
        'phone': phone.group() if phone else "Not found",
        'linkedin': linkedin.group() if linkedin else "Not found",
        'github': github.group() if github else "Not found"
    }

def clean_extracted_skills(raw_skills: List[str]) -> List[str]:
    """Enhanced skill normalization with more patterns and prioritization"""
    skill_patterns = [
        (r'python|pandas|numpy|scipy', 'Python'),
        (r'java|j2ee|spring boot', 'Java'),
        (r'c\+\+|cpp', 'C++'),
        (r'\bc\b(?!\+\+)', 'C'),
        (r'javascript|node\.?js|typescript', 'JavaScript'),
        (r'matlab', 'MATLAB'),
        (r'machine[\s-]?learning|ml\b', 'Machine Learning'),
        (r'data[\s-]?science', 'Data Science'),
        (r'data[\s-]?analysis', 'Data Analysis'),
        (r'deep[\s-]?learning', 'Deep Learning'),
        (r'supervised learning', 'Supervised Learning'),
        (r'unsupervised learning', 'Unsupervised Learning'),
        (r'feature engineering', 'Feature Engineering'),
        (r'object[\s-]?oriented|oop', 'Object-Oriented Programming'),
        (r'data structures', 'Data Structures'),
        (r'algorithms', 'Algorithms'),
        (r'embedded systems', 'Embedded Systems'),
        (r'iot|internet of things', 'IoT'),
        (r'arduino', 'Arduino'),
        (r'esp32', 'ESP32'),
        (r'adafruit', 'Adafruit IO'),
        (r'sensors?', 'Sensors'),
        (r'actuators?', 'Actuators'),
        (r'circuit design', 'Circuit Design'),
        (r'microcontrollers?', 'Microcontrollers'),
        (r'mysql', 'MySQL'),
        (r'oracle', 'Oracle'),
        (r'sql\b', 'SQL'),
        (r'tensorflow|tf\b', 'TensorFlow'),
        (r'pytorch', 'PyTorch'),
        (r'pandas', 'Pandas'),
        (r'numpy', 'NumPy'),
        (r'scikit[\s-]?learn|sklearn', 'Scikit-learn')
    ]
    
    normalized = set()
    for skill in raw_skills:
        text = skill.lower()
        for pattern, name in skill_patterns:
            if re.search(pattern, text):
                normalized.add(name)
                break
                
    # Prioritize more specific skills
    prioritized = sorted(normalized, key=lambda x: -SKILL_DATABASE.get(x, {}).get('weight', 1))
    return prioritized

def parallel_extract(text_chunk: str) -> List[str]:
    """Parallel processing helper for skill extraction with enhanced parameters"""
    try:
        keywords = KW_MODEL.extract_keywords(
            text_chunk, 
            keyphrase_ngram_range=(1, 3),
            top_n=10,
            stop_words='english',
            use_mmr=True,
            diversity=0.5
        )
        return [kw[0] for kw in keywords]
    except Exception as e:
        print(f"⚠️ Keyword extraction error: {e}")
        return []

def extract_skills(text: str, top_n: int = 20) -> List[str]:
    """Enhanced skill extraction with better chunking and fallback"""
    if not text:
        return []
        
    # Split into meaningful sections (education, experience, etc.)
    sections = re.split(r'\n\s*(?:Education|Experience|Skills|Projects)\s*\n', text, flags=re.IGNORECASE)
    sections = [s for s in sections if s.strip()]
    
    # Process each section separately
    with ThreadPoolExecutor() as executor:
        raw_skills = []
        for section in sections[:4]:  # Limit to first 4 sections to avoid too much processing
            chunks = [section[i:i+1500] for i in range(0, len(section), 1500)][:3]  # Larger chunks
            raw_skills.extend(sum(executor.map(parallel_extract, chunks), []))
    
    cleaned_skills = clean_extracted_skills(raw_skills)
    return cleaned_skills[:top_n]

def generate_summary(text: str) -> str:
    """Enhanced summary generation with fallback"""
    if not text:
        return "Unable to generate summary - no text extracted."
    
    try:
        # Try with first 1024 characters
        summary = SUMMARIZER(
            text[:1024],
            max_length=150,
            min_length=40,
            do_sample=False,
            truncation=True
        )[0]['summary_text']
        return summary
    except Exception as e:
        print(f"⚠️ Summary generation error: {e}")
        # Fallback to simple extraction of first few lines
        return " ".join(text.split('\n')[:3]).strip() + "..."

def analyze_skills(skills: List[str]) -> Dict:
    """Enhanced skill analysis with weighted scoring"""
    categories = {
        'Programming': [], 
        'AI/Data Science': [], 
        'Web Development': [],
        'Software Development': [],
        'Embedded/IoT': [], 
        'Databases': [], 
        'Scientific Computing': [],
        'Other': []
    }
    
    total_possible = 0
    weighted_score = 0
    
    for skill in skills:
        skill_data = SKILL_DATABASE.get(skill, {'category': 'Other', 'demand': 'Medium', 'weight': 1})
        categories[skill_data['category']].append(skill)
        
        # Calculate weighted score
        demand_score = {
            'Very High': 4,
            'High': 3,
            'Medium': 2,
            'Low': 1
        }.get(skill_data['demand'], 2)
        
        weighted_score += demand_score * skill_data.get('weight', 1)
        total_possible += 4 * skill_data.get('weight', 1)  # Max possible per skill
    
    # Calculate percentage score with normalization
    if total_possible > 0:
        normalized_score = int((weighted_score / total_possible) * 100)
    else:
        normalized_score = 0
    
    # Calculate category percentages
    category_percentages = {
        cat: len(skills) / len(skills) * 100 if skills else 0
        for cat, skills in categories.items()
    }
    
    return {
        'categorized': categories,
        'score': normalized_score,
        'skill_counts': Counter(skills),
        'category_percentages': category_percentages,
        'top_skills': [s for s in skills if s in SKILL_DATABASE][:5]
    }

def fetch_live_jobs(skills: List[str], location: str = "") -> List[Dict]:
    """
    Enhanced job fetching with multiple API fallbacks and better error handling
    """
    if not skills:
        return []
    
    # Try multiple APIs in sequence
    apis_to_try = [
        try_remotive_api,
        try_reed_api,
        try_fallback_jobs
    ]
    
    for api in apis_to_try:
        try:
            jobs = api(skills, location)
            if jobs:
                return jobs
        except Exception as e:
            print(f"⚠️ API error ({api.__name__}): {e}")
            continue
    
    return []

def try_remotive_api(skills: List[str], location: str) -> List[Dict]:
    """Try Remotive API with enhanced parameters"""
    query = "+".join(skills[:3])
    url = f"https://remotive.io/api/remote-jobs?search={query}&limit=5"
    
    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        return process_job_data(resp.json().get('jobs', []))
    except requests.exceptions.SSLError:
        # Retry without SSL verification
        resp = requests.get(url, timeout=15, verify=False)
        resp.raise_for_status()
        return process_job_data(resp.json().get('jobs', []))

def try_reed_api(skills: List[str], location: str) -> List[Dict]:
    """Alternative API source (mock implementation)"""
    # Note: This is a placeholder - you would need actual API credentials
    print("ℹ️ Trying Reed API...")
    return []

def process_job_data(jobs_list: List[Dict]) -> List[Dict]:
    """Enhanced job data processing with more fields"""
    processed = []
    for job in jobs_list[:5]:  # Limit to 5 jobs
        processed.append({
            "title": job.get('title', 'N/A').strip(),
            "company": job.get('company_name', 'N/A').strip(),
            "type": job.get('job_type', 'N/A').capitalize(),
            "location": job.get('candidate_required_location', 'Remote').strip(),
            "salary": job.get('salary', 'Not disclosed').strip(),
            "url": job.get('url', '#').strip(),
            "source": "Remotive",
            "posted": format_posted_date(job.get('publication_date', '')),
            "match_score": calculate_enhanced_match_score(job),
            "description": job.get('description', '')[:200] + '...' if job.get('description') else ''
        })
    return processed

def format_posted_date(date_str: str) -> str:
    """Format date string for display"""
    if not date_str:
        return "Recently"
    
    try:
        date_obj = datetime.strptime(date_str[:10], '%Y-%m-%d')
        delta = datetime.now() - date_obj
        
        if delta.days == 0:
            return "Today"
        elif delta.days == 1:
            return "Yesterday"
        elif delta.days < 7:
            return f"{delta.days} days ago"
        elif delta.days < 30:
            weeks = delta.days // 7
            return f"{weeks} week{'s' if weeks > 1 else ''} ago"
        else:
            return date_str[:10]
    except:
        return date_str[:10] if date_str else "Recently"

def calculate_enhanced_match_score(job: Dict) -> int:
    """More sophisticated match score calculation"""
    base_score = random.randint(65, 80)  # More realistic base range
    
    # Salary bonus
    if job.get('salary') and job['salary'].lower() not in ('not disclosed', 'competitive'):
        base_score += 10
        
    # Job type bonus
    job_type = job.get('job_type', '').lower()
    if 'full' in job_type:
        base_score += 5
    elif 'part' in job_type:
        base_score -= 5
        
    # Location bonus
    location = job.get('candidate_required_location', '').lower()
    if 'remote' in location:
        base_score += 8
    elif 'hybrid' in location:
        base_score += 4
        
    # Freshness bonus (more recent = higher score)
    posted_date = job.get('publication_date', '')
    if posted_date:
        try:
            days_old = (datetime.now() - datetime.strptime(posted_date[:10], '%Y-%m-%d')).days
            if days_old < 7:
                base_score += 5
            elif days_old < 14:
                base_score += 3
        except:
            pass
            
    return min(max(base_score, 0), 100)  # Ensure score is between 0-100

def try_fallback_jobs(skills: List[str], location: str) -> List[Dict]:
    """Enhanced fallback job source with more realistic data"""
    print("ℹ️ Using enhanced fallback job data...")
    
    # Skill-based job titles
    primary_skill = skills[0] if skills else "Software"
    secondary_skill = skills[1] if len(skills) > 1 else "Data"
    
    fallback_jobs = [
        {
            "title": f"{primary_skill} Developer",
            "company": "Tech Innovations Inc.",
            "type": "Full-time",
            "location": "Remote",
            "salary": "$85,000 - $120,000",
            "url": "https://example.com/jobs/123",
            "source": "Fallback",
            "posted": (datetime.now() - timedelta(days=random.randint(0, 3))).strftime('%Y-%m-%d'),
            "match_score": random.randint(75, 90),
            "description": f"Seeking a skilled {primary_skill} developer with experience in {secondary_skill}."
        },
        {
            "title": f"Senior {secondary_skill} Engineer",
            "company": "Data Systems Co.",
            "type": "Full-time",
            "location": "Hybrid (NYC)",
            "salary": "$95,000 - $135,000",
            "url": "https://example.com/jobs/456",
            "source": "Fallback",
            "posted": (datetime.now() - timedelta(days=random.randint(1, 5))).strftime('%Y-%m-%d'),
            "match_score": random.randint(70, 85),
            "description": f"Senior position requiring expertise in {secondary_skill} and {primary_skill}."
        },
        {
            "title": f"{primary_skill} Specialist",
            "company": "Digital Solutions LLC",
            "type": "Contract",
            "location": "Remote",
            "salary": "$50 - $75/hour",
            "url": "https://example.com/jobs/789",
            "source": "Fallback",
            "posted": (datetime.now() - timedelta(days=random.randint(0, 2))).strftime('%Y-%m-%d'),
            "match_score": random.randint(65, 80),
            "description": f"Contract position for {primary_skill} implementation."
        }
    ]
    
    return fallback_jobs[:3]

def generate_certificates_section(skills: List[str] = None) -> str:
    """Generate dynamic certificates section based on skills"""
    base_certs = [
        "Data Analyst Course",
        "Machine Learning with Python (Coursera)",
        "Circuit Design & Microcontroller Integration",
        "Database Management Fundamentals"
    ]
    
    skill_certs = []
    if skills:
        if any(s in skills for s in ['Python', 'Data Science']):
            skill_certs.append("Python for Data Science Certification")
        if 'Machine Learning' in skills:
            skill_certs.append("Advanced Machine Learning Specialization")
        if 'Embedded Systems' in skills:
            skill_certs.append("Embedded Systems Design Certification")
    
    all_certs = base_certs + skill_certs
    cert_items = "\n".join(f"• {cert}" for cert in all_certs)
    
    return f"Certificates:\n{cert_items}"

# ========== Modern GUI Application ==========
class ResumeAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Resume Analyzer Pro")
        self.root.geometry("1100x800")
        self.root.configure(bg=LIGHT_GRAY)
        
        # Configure style
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Configure colors
        self.style.configure('TFrame', background=LIGHT_GRAY)
        self.style.configure('TLabel', background=LIGHT_GRAY, foreground=DARK_GRAY, font=FONT_MEDIUM)
        self.style.configure('TButton', background=LIGHT_BLUE, foreground=WHITE, font=FONT_MEDIUM,
                           borderwidth=1, focusthickness=3, focuscolor='none')
        self.style.map('TButton', 
                      background=[('active', DARK_BLUE), ('pressed', DARK_BLUE)],
                      foreground=[('active', WHITE), ('pressed', WHITE)])
        self.style.configure('TNotebook', background=LIGHT_GRAY)
        self.style.configure('TNotebook.Tab', background=LIGHT_BLUE, foreground=WHITE, 
                           font=FONT_MEDIUM, padding=[10, 5])
        self.style.map('TNotebook.Tab', 
                      background=[('selected', DARK_BLUE)],
                      foreground=[('selected', WHITE)])
        
        # Create widgets
        self.create_widgets()
        
        # Initialize analysis variables
        self.current_file = None
        self.analysis_results = None
        self.skills_figure = None
        
    def create_widgets(self):
        """Create all GUI widgets with modern design"""
        # Header Frame with gradient effect
        header_frame = tk.Frame(self.root, bg=DARK_BLUE, height=120)
        header_frame.pack(fill=tk.X)
        
        # Title Label
        title_label = tk.Label(
            header_frame,
            text="AI Powered Resume Analyzer and Job Matching",
            font=FONT_TITLE,
            fg=WHITE,
            bg=DARK_BLUE,
            pady=20
        )
        title_label.pack()
        
        # Subtitle Label
        subtitle_label = tk.Label(
            header_frame,
            text="Upload your resume to get personalized career insights",
            font=FONT_SUBTITLE,
            fg=ACCENT_BLUE,
            bg=DARK_BLUE
        )
        subtitle_label.pack()
        
        # Control Frame
        control_frame = ttk.Frame(self.root, padding="15")
        control_frame.pack(fill=tk.X)
        
        # File Upload Section
        upload_frame = ttk.Frame(control_frame)
        upload_frame.pack(side=tk.LEFT, padx=10)
        
        # Upload Button with icon
        self.upload_btn = ttk.Button(
            upload_frame,
            text="📄 Upload Resume",
            command=self.upload_resume,
            style='TButton'
        )
        self.upload_btn.pack(side=tk.LEFT)
        
        # File Label
        self.file_label = ttk.Label(
            upload_frame,
            text="No file selected",
            foreground=ACCENT_BLUE,
            font=FONT_SMALL
        )
        self.file_label.pack(side=tk.LEFT, padx=10)
        
        # Location Entry
        location_frame = ttk.Frame(control_frame)
        location_frame.pack(side=tk.LEFT, padx=10)
        
        self.location_label = ttk.Label(
            location_frame,
            text="📍 Location:",
            font=FONT_MEDIUM
        )
        self.location_label.pack(side=tk.LEFT)
        
        self.location_entry = ttk.Entry(
            location_frame,
            width=25,
            font=FONT_MEDIUM
        )
        self.location_entry.pack(side=tk.LEFT, padx=5)
        self.location_entry.insert(0, "Remote")
        
        # Analyze Button
        self.analyze_btn = ttk.Button(
            control_frame,
            text="🔍 Analyze Resume",
            command=self.start_analysis,
            state=tk.DISABLED,
            style='TButton'
        )
        self.analyze_btn.pack(side=tk.RIGHT, padx=10)
        
        # Progress Section
        progress_frame = ttk.Frame(self.root)
        progress_frame.pack(fill=tk.X, pady=(0, 10))
        
        # Progress Bar
        self.progress = ttk.Progressbar(
            progress_frame,
            orient=tk.HORIZONTAL,
            length=500,
            mode='determinate',
            style='TProgressbar'
        )
        self.progress.pack(pady=5)
        
        # Status Label
        self.status_label = ttk.Label(
            progress_frame,
            text="Ready to analyze",
            foreground=DARK_BLUE,
            font=FONT_SMALL
        )
        self.status_label.pack()
        
        # Notebook for Results
        self.notebook = ttk.Notebook(self.root, style='TNotebook')
        self.notebook.pack(fill=tk.BOTH, expand=True, padx=15, pady=(0, 15))
        
        # Create tabs
        self.create_tabs()
        
    def create_tabs(self):
        """Create the tabs for the notebook with modern styling"""
        # Overview Tab
        self.overview_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.overview_tab, text="📋 Overview")
        
        # Skills Tab
        self.skills_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.skills_tab, text="🛠 Skills Analysis")
        
        # Jobs Tab
        self.jobs_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.jobs_tab, text="💼 Job Matches")
        
        # Recommendations Tab
        self.recommendations_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.recommendations_tab, text="💡 Recommendations")
        
        # Initialize tab content
        self.init_tab_content()
        
    def init_tab_content(self):
        """Initialize empty content for tabs with modern styling"""
        # Overview Tab
        self.overview_text = scrolledtext.ScrolledText(
            self.overview_tab,
            wrap=tk.WORD,
            font=FONT_MEDIUM,
            padx=15,
            pady=15,
            bg=WHITE,
            fg=DARK_GRAY,
            insertbackground=ACCENT_BLUE,
            selectbackground=ACCENT_BLUE,
            selectforeground=WHITE,
            relief=tk.FLAT
        )
        self.overview_text.pack(fill=tk.BOTH, expand=True)
        self.overview_text.insert(tk.END, "Upload and analyze a resume to see results here.")
        self.overview_text.config(state=tk.DISABLED)
        
        # Skills Tab
        self.skills_canvas_frame = ttk.Frame(self.skills_tab)
        self.skills_canvas_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Jobs Tab
        self.jobs_canvas = tk.Canvas(self.jobs_tab, bg=LIGHT_GRAY, highlightthickness=0)
        self.jobs_scrollbar = ttk.Scrollbar(self.jobs_tab, orient=tk.VERTICAL, command=self.jobs_canvas.yview)
        self.jobs_scrollable_frame = ttk.Frame(self.jobs_canvas)
        
        self.jobs_scrollable_frame.bind(
            "<Configure>",
            lambda e: self.jobs_canvas.configure(
                scrollregion=self.jobs_canvas.bbox("all")
            )
        )
        
        self.jobs_canvas.create_window((0, 0), window=self.jobs_scrollable_frame, anchor="nw")
        self.jobs_canvas.configure(yscrollcommand=self.jobs_scrollbar.set)
        
        self.jobs_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.jobs_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Recommendations Tab
        self.recommendations_text = scrolledtext.ScrolledText(
            self.recommendations_tab,
            wrap=tk.WORD,
            font=FONT_MEDIUM,
            padx=15,
            pady=15,
            bg=WHITE,
            fg=DARK_GRAY,
            insertbackground=ACCENT_BLUE,
            selectbackground=ACCENT_BLUE,
            selectforeground=WHITE,
            relief=tk.FLAT
        )
        self.recommendations_text.pack(fill=tk.BOTH, expand=True)
        self.recommendations_text.insert(tk.END, "Personalized recommendations will appear here after analysis.")
        self.recommendations_text.config(state=tk.DISABLED)
        
    def upload_resume(self):
        """Handle resume file upload with modern file dialog"""
        file_path = filedialog.askopenfilename(
            title="Select Resume PDF",
            filetypes=[("PDF Files", "*.pdf"), ("All Files", "*.*")],
            initialdir=os.path.expanduser("~")
        )
        
        if file_path:
            self.current_file = file_path
            short_name = os.path.basename(file_path)
            if len(short_name) > 30:
                short_name = "..." + short_name[-27:]
            self.file_label.config(text=short_name)
            self.analyze_btn.config(state=tk.NORMAL)
            
    def start_analysis(self):
        """Start the analysis process in a separate thread"""
        if not self.current_file:
            messagebox.showerror("Error", "Please upload a resume first")
            return
            
        # Disable buttons during analysis
        self.upload_btn.config(state=tk.DISABLED)
        self.analyze_btn.config(state=tk.DISABLED)
        self.progress["value"] = 0
        self.status_label.config(text="Starting analysis...")
        
        # Start analysis in a separate thread
        analysis_thread = threading.Thread(
            target=self.perform_analysis,
            args=(self.current_file, self.location_entry.get()),
            daemon=True
        )
        analysis_thread.start()
        
        # Check progress periodically
        self.root.after(100, self.check_progress)
        
    def perform_analysis(self, pdf_path: str, location: str):
        """Perform the actual resume analysis"""
        try:
            # Initialize progress
            self.update_progress(10, "Extracting text from resume...")
            
            # Extract text
            text = extract_text_from_pdf(pdf_path)
            if not text.strip():
                raise ValueError("Empty resume text - possible extraction error")
            
            self.update_progress(20, "Analyzing resume content...")
            
            # Extract information
            with ThreadPoolExecutor() as pool:
                contact_future = pool.submit(extract_contact_info, text)
                skills_future = pool.submit(extract_skills, text)
                summary_future = pool.submit(generate_summary, text)
                
                contact = contact_future.result()
                skills = skills_future.result()
                summary = summary_future.result()
            
            self.update_progress(50, "Analyzing skills...")
            skill_data = analyze_skills(skills)
            
            self.update_progress(70, "Fetching job opportunities...")
            jobs = fetch_live_jobs(skills, location)
            
            self.update_progress(90, "Generating report...")
            
            # Store results
            self.analysis_results = {
                'contact': contact,
                'summary': summary,
                'skill_data': skill_data,
                'jobs': jobs,
                'skills': skills
            }
            
            # Update UI with results
            self.root.after(0, self.display_results)
            
            self.update_progress(100, "Analysis complete!")
            
        except Exception as e:
            self.root.after(0, lambda: messagebox.showerror("Analysis Error", f"Error during analysis: {e}"))
            self.update_progress(0, "Analysis failed")
            self.upload_btn.config(state=tk.NORMAL)
            self.analyze_btn.config(state=tk.NORMAL)
            
    def update_progress(self, value: int, message: str):
        """Update progress bar and status label"""
        self.progress["value"] = value
        self.status_label.config(text=message)
        
    def check_progress(self):
        """Check if analysis is complete"""
        if self.progress["value"] < 100:
            self.root.after(100, self.check_progress)
        else:
            self.upload_btn.config(state=tk.NORMAL)
            self.analyze_btn.config(state=tk.NORMAL)
            
    def display_results(self):
        """Display the analysis results in the UI"""
        if not self.analysis_results:
            return
            
        # Display overview
        self.display_overview()
        
        # Display skills analysis
        self.display_skills()
        
        # Display job opportunities
        self.display_jobs()
        
        # Display recommendations
        self.display_recommendations()
        
    def display_overview(self):
        """Display overview tab content with modern styling"""
        contact = self.analysis_results['contact']
        summary = self.analysis_results['summary']
        
        overview_text = f"""
        ╔════════════════════════════════════════════╗
        ║            CONTACT INFORMATION             ║
        ╚════════════════════════════════════════════╝
        
        📧 Email: {contact['email']}
        📞 Phone: {contact['phone']}
        🔗 LinkedIn: {contact['linkedin']}
        🐱 GitHub: {contact['github']}
        
        ╔════════════════════════════════════════════╗
        ║            PROFESSIONAL SUMMARY            ║
        ╚════════════════════════════════════════════╝
        
        {summary}
        
        ╔════════════════════════════════════════════╗
        ║               CERTIFICATES                 ║
        ╚════════════════════════════════════════════╝
        
        {generate_certificates_section(self.analysis_results['skills'])}
        """
        
        self.overview_text.config(state=tk.NORMAL)
        self.overview_text.delete(1.0, tk.END)
        
        # Configure tags for styling
        self.overview_text.tag_configure("header", foreground=DARK_BLUE, font=("Arial", 12, "bold"))
        self.overview_text.tag_configure("contact", foreground=ACCENT_BLUE)
        self.overview_text.tag_configure("section", foreground=DARK_BLUE, font=("Arial", 11, "bold"))
        
        # Insert text with tags
        self.overview_text.insert(tk.END, "CONTACT INFORMATION\n", "header")
        self.overview_text.insert(tk.END, f"\n📧 Email: {contact['email']}\n", "contact")
        self.overview_text.insert(tk.END, f"📞 Phone: {contact['phone']}\n", "contact")
        self.overview_text.insert(tk.END, f"🔗 LinkedIn: {contact['linkedin']}\n", "contact")
        self.overview_text.insert(tk.END, f"🐱 GitHub: {contact['github']}\n\n", "contact")
        
        self.overview_text.insert(tk.END, "PROFESSIONAL SUMMARY\n", "header")
        self.overview_text.insert(tk.END, f"\n{summary}\n\n", "section")
        
        self.overview_text.insert(tk.END, "CERTIFICATES\n", "header")
        self.overview_text.insert(tk.END, f"\n{generate_certificates_section(self.analysis_results['skills'])}", "section")
        
        self.overview_text.config(state=tk.DISABLED)
        
    def display_skills(self):
        """Display skills analysis with modern visualizations"""
        skill_data = self.analysis_results['skill_data']
        
        # Clear previous content
        for widget in self.skills_canvas_frame.winfo_children():
            widget.destroy()
            
        # Create figure with modern style
        try:
            plt.style.use('ggplot')  # Using ggplot style instead of seaborn
        except:
            plt.style.use('default')  # Fallback to default style if ggplot not available
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        fig.patch.set_facecolor(LIGHT_GRAY)
        
        # Skill Category Distribution
        sizes = [len(v) for v in skill_data['categorized'].values() if v]
        labels = [k for k, v in skill_data['categorized'].items() if v]
        colors = sns.color_palette("husl", len(labels))
        
        # Explode the largest slice
        if sizes:
            max_index = sizes.index(max(sizes))
            explode = [0.1 if i == max_index else 0 for i in range(len(sizes))]
            ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', 
                   startangle=90, explode=explode, shadow=True)
            ax1.set_title('Skill Category Distribution', pad=20, fontweight='bold')
        else:
            ax1.text(0.5, 0.5, 'No skills detected', ha='center', va='center')
            ax1.set_title('No Skills Found', pad=20)
        
        # Top Skills
        top_skills = skill_data['skill_counts'].most_common(8)
        if top_skills:
            sns.barplot(
                x=[x[1] for x in top_skills], 
                y=[x[0] for x in top_skills], 
                palette='viridis',
                ax=ax2
            )
            ax2.set_title('Top Skills by Frequency', pad=20, fontweight='bold')
            ax2.set_xlabel('Mentions', fontweight='bold')
            ax2.set_ylabel('')
        else:
            ax2.text(0.5, 0.5, 'No skills detected', ha='center', va='center')
            ax2.set_title('No Skills Found', pad=20)
        
        plt.tight_layout()
        
        # Embed figure in Tkinter
        canvas = FigureCanvasTkAgg(fig, master=self.skills_canvas_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Score display with color coding
        score = skill_data['score']
        if score >= 80:
            color = GREEN
            feedback = "Excellent!"
        elif score >= 60:
            color = ACCENT_BLUE
            feedback = "Good"
        elif score >= 40:
            color = ORANGE
            feedback = "Average"
        else:
            color = RED
            feedback = "Needs Improvement"
            
        score_frame = ttk.Frame(self.skills_canvas_frame)
        score_frame.pack(pady=(0, 10))
        
        score_label = ttk.Label(
            score_frame,
            text="Skills Score:",
            font=("Segoe UI", 12),
            foreground=DARK_GRAY
        )
        score_label.pack(side=tk.LEFT)
        
        score_value = ttk.Label(
            score_frame,
            text=f"{score}/100 ({feedback})",
            font=("Segoe UI", 12, "bold"),
            foreground=color
        )
        score_value.pack(side=tk.LEFT, padx=5)
        
        # Store figure reference
        self.skills_figure = fig
        
    def display_jobs(self):
        """Display job opportunities with modern cards"""
        jobs = self.analysis_results['jobs']
        
        # Clear previous content
        for widget in self.jobs_scrollable_frame.winfo_children():
            widget.destroy()
            
        if not jobs:
            no_jobs_frame = ttk.Frame(self.jobs_scrollable_frame, style='TFrame')
            no_jobs_frame.pack(fill=tk.X, pady=20, padx=20)
            
            no_jobs_label = ttk.Label(
                no_jobs_frame,
                text="No relevant job opportunities found.\n\nTry:\n• Expanding your skill set\n• Broadening your search criteria\n• Checking back later for updates",
                font=("Segoe UI", 12),
                justify=tk.CENTER,
                foreground=DARK_GRAY
            )
            no_jobs_label.pack(pady=20)
            return
            
        for job in jobs:
            match_color = GREEN if job['match_score'] > 75 else ORANGE if job['match_score'] > 60 else RED
            
            # Main job card frame
            job_frame = tk.Frame(
                self.jobs_scrollable_frame,
                bg=WHITE,
                bd=0,
                highlightbackground="#e0e0e0",
                highlightthickness=1,
                relief=tk.RAISED
            )
            job_frame.pack(fill=tk.X, pady=10, padx=20, ipady=10, ipadx=5)
            
            # Title and Company
            title_frame = tk.Frame(job_frame, bg=WHITE)
            title_frame.pack(fill=tk.X, pady=(5, 0), padx=10)
            
            tk.Label(
                title_frame,
                text=job['title'],
                font=("Segoe UI", 14, "bold"),
                bg=WHITE,
                fg=DARK_BLUE
            ).pack(side=tk.LEFT)
            
            tk.Label(
                title_frame,
                text=f"at {job['company']}",
                font=("Segoe UI", 12),
                bg=WHITE,
                fg=DARK_GRAY
            ).pack(side=tk.LEFT, padx=5)
            
            # Job Details
            details_frame = tk.Frame(job_frame, bg=WHITE)
            details_frame.pack(fill=tk.X, pady=5, padx=10)
            
            # Left details (type, location)
            left_frame = tk.Frame(details_frame, bg=WHITE)
            left_frame.pack(side=tk.LEFT, fill=tk.X, expand=True)
            
            tk.Label(
                left_frame,
                text=f"🕒 {job['type']}",
                font=("Segoe UI", 10),
                bg=WHITE,
                fg=DARK_GRAY
            ).pack(anchor=tk.W)
            
            tk.Label(
                left_frame,
                text=f"📍 {job['location']}",
                font=("Segoe UI", 10),
                bg=WHITE,
                fg=DARK_GRAY
            ).pack(anchor=tk.W)
            
            # Right details (salary, posted)
            right_frame = tk.Frame(details_frame, bg=WHITE)
            right_frame.pack(side=tk.RIGHT, fill=tk.X)
            
            tk.Label(
                right_frame,
                text=f"💰 {job['salary']}",
                font=("Segoe UI", 10),
                bg=WHITE,
                fg=DARK_GRAY
            ).pack(anchor=tk.E)
            
            tk.Label(
                right_frame,
                text=f"📅 {job['posted']}",
                font=("Segoe UI", 10),
                bg=WHITE,
                fg=DARK_GRAY
            ).pack(anchor=tk.E)
            
            # Match score
            match_frame = tk.Frame(job_frame, bg=WHITE)
            match_frame.pack(fill=tk.X, pady=5, padx=10)
            
            tk.Label(
                match_frame,
                text="Match Score:",
                font=("Segoe UI", 10),
                bg=WHITE,
                fg=DARK_GRAY
            ).pack(side=tk.LEFT)
            
            tk.Label(
                match_frame,
                text=f"{job['match_score']}%",
                font=("Segoe UI", 10, "bold"),
                bg=WHITE,
                fg=match_color
            ).pack(side=tk.LEFT, padx=5)
            
            # Description
            desc_frame = tk.Frame(job_frame, bg=WHITE)
            desc_frame.pack(fill=tk.X, pady=5, padx=10)
            
            desc_text = tk.Text(
                desc_frame,
                wrap=tk.WORD,
                height=4,
                font=("Segoe UI", 10),
                padx=5,
                pady=5,
                bg=WHITE,
                fg=DARK_GRAY,
                relief=tk.FLAT,
                highlightbackground="#e0e0e0",
                highlightthickness=1
            )
            desc_text.insert(tk.END, job['description'])
            desc_text.config(state=tk.DISABLED)
            desc_text.pack(fill=tk.X)
            
            # Button and source frame
            button_frame = tk.Frame(job_frame, bg=WHITE)
            button_frame.pack(fill=tk.X, pady=(5, 10), padx=10)
            
            # Apply Button
            apply_btn = tk.Button(
                button_frame,
                text="🔗 Apply Now",
                command=lambda url=job['url']: self.open_url(url),
                bg=ACCENT_BLUE,
                fg=WHITE,
                activebackground=DARK_BLUE,
                activeforeground=WHITE,
                relief=tk.FLAT,
                font=("Segoe UI", 10, "bold"),
                padx=15,
                pady=5
            )
            apply_btn.pack(side=tk.LEFT)
            
            # Source label
            tk.Label(
                button_frame,
                text=f"Source: {job['source']}",
                font=("Segoe UI", 8),
                bg=WHITE,
                fg=DARK_GRAY
            ).pack(side=tk.RIGHT)
            
    def display_recommendations(self):
        """Display career recommendations with modern styling"""
        skill_data = self.analysis_results['skill_data']
        jobs = self.analysis_results['jobs']
        
        recommendations = []
        
        # Skill-based recommendations
        if skill_data['score'] < 60:
            recommendations.append("• Consider adding more in-demand skills to your resume")
        if len(skill_data['top_skills']) < 3:
            recommendations.append("• Highlight your top skills more prominently in your resume")
        if not jobs:
            recommendations.append("• Try networking on LinkedIn to discover unadvertised opportunities")
            
        # Add general recommendations if none specific
        if not recommendations:
            recommendations = [
                "• Your resume looks strong! Consider tailoring it for specific job applications",
                "• Keep your LinkedIn profile updated with similar information",
                "• Consider adding quantifiable achievements to your resume"
            ]
            
        # Format recommendations
        rec_text = "🌟 Career Recommendations\n\n" + "\n".join(recommendations)
        
        self.recommendations_text.config(state=tk.NORMAL)
        self.recommendations_text.delete(1.0, tk.END)
        
        # Configure tags for styling
        self.recommendations_text.tag_configure("header", foreground=DARK_BLUE, font=("Arial", 14, "bold"))
        self.recommendations_text.tag_configure("bullet", foreground=ACCENT_BLUE, font=("Arial", 12))
        
        # Insert text with tags
        self.recommendations_text.insert(tk.END, "Career Recommendations\n", "header")
        self.recommendations_text.insert(tk.END, "\n" + "\n".join(recommendations), "bullet")
        
        self.recommendations_text.config(state=tk.DISABLED)
        
    def open_url(self, url):
        """Open URL in default browser"""
        import webbrowser
        webbrowser.open(url)
        
    def on_closing(self):
        """Handle window closing"""
        # Clean up matplotlib figures
        if self.skills_figure:
            plt.close(self.skills_figure)
        self.root.destroy()

# ========== Main Application ==========
if __name__ == "__main__":
    root = tk.Tk()
    
    # Set window icon (replace with your own icon if desired)
    try:
        root.iconbitmap("resume_icon.ico")  # Provide your own icon file
    except:
        pass  # Use default icon if custom one not available
    
    app = ResumeAnalyzerApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()


⚠️ API error (try_remotive_api): 526 Server Error: <none> for url: https://remotive.io/api/remote-jobs?search=Machine%20Learning+Python+Feature%20Engineering&limit=5
ℹ️ Trying Reed API...
ℹ️ Using enhanced fallback job data...
